In [1]:
%matplotlib inline
import numpy as np
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
from netCDF4 import date2index
import datetime as dt
import pandas as pd
from StringIO import StringIO
import xarray as xr

/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
import re

def extractURLsatellite(fileURL, satName):
    """
    Function to extract the URLs for a specific satellite from the IMOS URLs list
    
    Parameters:
    -----------
    
    - fileURL : IMOS URLs list as a txt file
    - satName : name of the satellite such as JASON-2 JASON-3 
    
    Ouputs:
    ------
    
    - getFiles : list of URLs for the desired satellite
    
    """
    
    getFiles = []
    
    with open(fileURL) as f:
        for line in f:
            if re.search(r"%s"%satName, line):
                changeURL = re.sub('http://data.aodn.org.au', 'http://thredds.aodn.org.au/thredds/dodsC', line)
                getFiles.append(changeURL)
                
    return getFiles

In [3]:
jason2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-2')

jason3URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-3')

saralURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SARAL')

sentinel3aURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SENTINEL-3A')

cryosat2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'CRYOSAT-2')

enviURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ENVISAT')

geosatURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GEOSAT')

ersURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ERS-2')

gfoURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GFO')

topURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'TOPEX')

In [4]:
allURL = []
allURL.append(jason2URL)
allURL.append(jason3URL)
allURL.append(saralURL)
allURL.append(sentinel3aURL)
allURL.append(cryosat2URL)
allURL.append(enviURL)
allURL.append(geosatURL)
allURL.append(ersURL)
allURL.append(gfoURL)
allURL.append(topURL)

In [5]:
boxLat = [] #latitude
boxLon = [] #longitude
boxWh = [] #wave height
boxT = [] #time
boxQ = [] #quality control flag
boxB = [] #backscatter
boxU = [] #wind Speed

In [6]:
#Cape Melville coordinates
latmin = -14.0
latmax = -13.0  
lonmin = 145.0
lonmax = 146.0

if latmin>latmax:
    print('Error wrong definition of min and max lat!!!')

if lonmin>lonmax:
    print('Error wrong definition of min and max lon!!!')

In [7]:
# 
start_date = dt.datetime(1985,1,1)
end_date = dt.datetime(2019,2,21)

#start_date = dt.datetime(2010,1,1)
#end_date = dt.datetime(2010,12,31)

In [8]:
for u in range (len(allURL)):
    urlON = allURL[u]
    for k in range (len(urlON)):
        ncs = NetCDFFile(urlON[k])
        lats = ncs.variables['LATITUDE'][:]
        lons = ncs.variables['LONGITUDE'][:]
        ws = ncs.variables['WSPD_CAL'][:]
       
        if u == 2:
            wh = ncs.variables['SWH_KA_CAL'][:]
            qc = ncs.variables['SWH_KA_quality_control'][:]
            back = ncs.variables['SIG0_KA'][:]
            
        else:
            wh = ncs.variables['SWH_KU_CAL'][:]
            qc = ncs.variables['SWH_KU_quality_control'][:]
            back = ncs.variables['SIG0_KU'][:]

            
        # Get desired time interval  
        time_var = ncs.variables['TIME']
        tt = ncs.variables['TIME'][:]
        
        timing = netCDF4.num2date(tt,time_var.units)
        
        #data in correct time, quality flag and lat/lon
        for k in range(len(timing)):
            if timing[k] >= start_date and timing[k] <= end_date:
                if lats[k]>latmin and lats[k]<latmax and lons[k]>lonmin and lons[k]<lonmax:
                    boxLat.append(lats[k])
                    boxLon.append(lons[k])
                    boxWh.append(wh[k])
                    boxT.append(tt[k])
                    boxQ.append(qc[k])
                    boxB.append(back[k])
                    boxU.append(ws[k])

In [9]:
len(boxT)

36666

In [11]:
for k in range(len(boxLat)):
    if k == 0:
        lat = boxLat[k]
        lon = boxLon[k]
        wh = boxWh[k]
        tt = boxT[k]
        qc = boxQ[k]
        back = boxB[k]
        ws = boxU[k]
    else:
        lat = np.append(lat,boxLat[k])
        lon = np.append(lon,boxLon[k])
        wh = np.append(wh,boxWh[k])
        tt = np.append(tt,boxT[k])
        qc = np.append(qc,boxQ[k])
        back = np.append(back,boxB[k])
        ws = np.append(ws, boxU[k])


In [16]:
print len(wh)

36666


In [15]:
ids = np.where(np.logical_and(wh>0,qc==1))

lat_ = lat[ids]
lon_ = lon[ids]
whpos = wh[ids]
tt_ = tt[ids]
qc_ = qc[ids]
back_ = back[ids]
ws_ = ws[ids]

print len(whpos), 'whpos'


34802 whpos


In [19]:
print 'mean whpos', np.mean(whpos)
print 'max whpos',max(whpos)
print 'min whpos', min (whpos)

print 'min lon',min(lon_)
print 'max lon',max(lon_)

print 'min lat', min(lat_)
print 'max lat',max(lat_)

print 'min backscatter', min(back_)
print 'max backscatter', max(back_)

print 'min ws', min(ws_)
print 'max ws', max(ws_)

print 'start time', min(netCDF4.num2date(tt_,time_var.units))
print 'end time', max (netCDF4.num2date(tt_,time_var.units))

mean whpos 1.6800049060284783
max whpos 4.949000358581543
min whpos 0.003000000026077032
min lon 145.00002
max lon 145.99998
min lat -13.999892
max lat -13.000028
min backscatter 4.79
max backscatter 28.01
min ws 0.5600000023841858
max ws 39.36000061035156
start time 1985-04-05 21:27:22.624997
end time 2019-02-20 11:53:11.406250


In [ ]:
for k in range(len(whpos)):
    #if np.where(np.logical_and(wh>0,qc==1)):
    print netCDF4.num2date(whpos[k],time_var.units)

In [ ]:
for k in range(len(whpos)):
    #if np.where(np.logical_and(wh>0,qc==1)):
    print netCDF4.num2date(tt_[k],time_var.units)

In [ ]:
sorted1 = np.sort(tt_)
for k in range(len(sorted1)):
    print netCDF4.num2date(sorted1[k],time_var.units)

WAVE PERIOD EQN

$ \epsilon = 3.25 \left( H_s^2 g^2/U^4 \right)^{0.31}$



wave age. g = 9.80665 m/s2.  ξ=3.25(H2sg2/U4)^0.31.

In [25]:
np.power(whpos,4)

masked_array(data=[19.04380601, 16.61672076, 13.36446587, ...,
                   11.33820514, 11.33820514,  8.93675939],
             mask=False,
       fill_value=1e+20)

In [36]:
def waveage(H, U, grav=9.80665):
    '''
    The pseudo wave age can be expressed in terms of significant wave height and surface wind speed.
    '''

    grav2 = grav**2
    hs2 = np.square(H) 
    u4 = np.power(U,4)
    tmp = np.divide(hs2*grav2,u4)
    eps = 3.25*np.power(tmp,0.31)
    
    return eps


def waveperiod(H, U, grav=9.80665):
    '''
    blah blah
    '''

    eps = waveage(H, U, grav=9.80665)
    period = (((eps-(5.78))/(eps+(U/(H*((U/H)+H)))))+(H+(5.70)))
    
    return period

In [38]:
T = waveperiod(whpos,ws_)
print T
print len(T)

[5.38329868599818 5.243681379130724 5.135587000064403 ...
 5.458391830570296 5.333780063288117 5.1881342515098074]
34802


GA-2

In [ ]:
tz = (((w_a-(5.78))/(w_a+(ws_/(whpos*((ws_/whpos)+whpos)))))+(whpos+(5.7)))